# Função PWM e PSSM




As funções, **PWM** e **PSSM**, são ferramentas essenciais na análise de dados genômicos e na identificação de elementos regulatórios importantes nas sequências do ADN.

> ### Função PWM

A **função PWM** (Position Weight Matrix) é uma técnica amplamente utilizada na bioinformática para modelar padrões de consenso em sequências biológicas, como DNA ou proteínas. 

Consiste numa matriz que representa a frequência relativa de cada base em cada posição ao longo de um conjunto de sequências alinhadas.

É especialmente útil na identificação de **motifs** conservados em regiões regulatórias do DNA.

In [ ]:
'''
A instação da biblioteca *tabulate* permite a utilização de um conjunto de ferramentas de forma a ter um output mais organizado
'''

# pip install tabulate

In [45]:
'''
O *import* do módulo tabulate irá permitir a utilização das funções e classes fornecidas pelo *tabulate* para formatar e exibir dados tabulares
'''
# from tabulate import tabulate

seqs = ['TGACTATACGTATGGTAGAT', 'ATCGTATACGTAGGTAGAC', 'TAGCTAGTCGTATGGTAGAT']
pseudo = 0

def pwm(seqs: list[str], pseudo: float = 0) -> list[dict[str, float]]:
  alfabeto = 'ACGT'

  """
  Calcula a matriz PWM (Matriz de Peso e Posição) para as sequências fornecidas

  Parâmetros
  -------------
  seqs : list[str]
      Recebe uma lista de strings que representam as sequências

  pseudo : float = 0
      Recebe um valor opcional, pseudo, que em caso de omissão é = 0


  Retorna
  -------------
  pwm_matrix : list[dict[str, float]]
      Retorna uma *lista de dicionários*, onde cada *dicionário* terá uma chave no formato de *string*, e um valor no formato de *float*

  """

  for seq in seqs:
    for idx, b in enumerate(seq):
      assert b in alfabeto, f'Caracter {b} na posição {idx} da sequência {seq} inválido!'

  pwm_matrix = [{b: (pos.count(b) + pseudo) / (len(seqs) + len(alfabeto) * pseudo)
    for b in alfabeto}
      for pos in zip(*seqs)]

  return pwm_matrix

resultado = pwm(seqs, pseudo)  # Associação do resultado da função a uma variável de forma a ser mais fácil imprimir o resultado

# Impressão da matriz formatada com o módulo tabulate
headers = ["Base"] + [f"Posição {i+1}" for i in range(len(seqs[0]))]
table = [[base] + [round(resultado[i][base], 3) for i in range(len(resultado))] for base in "ACGT"]
print(tabulate(table, headers))

Base      Posição 1    Posição 2    Posição 3    Posição 4    Posição 5    Posição 6    Posição 7    Posição 8    Posição 9    Posição 10    Posição 11    Posição 12    Posição 13    Posição 14    Posição 15    Posição 16    Posição 17    Posição 18    Posição 19
------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------
A             0.333        0.333        0.333        0                0            1        0            0.667            0             0             0             1         0                 0         0             0.333         0.667         0.333         0.667
C             0            0            0.333        0.667            0            0        0            0                1             0             0             0         0                 0         0     

In [46]:
def prob_seq(sequence, resultado):
    """
    Calcula a probabilidade de uma sequência utilizando como base a função PWM

    Parâmetros
    -------------
    sequence : str 
        A sequência de DNA 
     
    resultado : list[dict[str, float]] 
        Variável que contém o resultado da função PWM, onde cada dicionário representa as probabilidades para cada base em uma posição

    Retorna
    -------------
    probabilidade : float 
        A probabilidade da sequência com base na PWM
    
    """
    
    probabilidade = 1.0

    for position, base in enumerate(sequence):
        if base in resultado[position]:
            probabilidade *= resultado[position][base]
        else:
            probabilidade *= 0.01  # Atribui uma probabilidade mínima de 0.01 para bases ausentes.

    return probabilidade

seq = 'ATA'
probabilidade = prob_seq(seq, resultado)

print(f"A probabilidade da sequência '{seq}' é de: \n{probabilidade}")

A probabilidade da sequência 'ATA' é de: 
0.037037037037037035


In [67]:
# import re

def seq_provavel(seq, resultado):
  
  '''
  Calcula qual a Sequência mais provável

  Parâmetros
  -------------
  seq : str
    A sequência de DNA
  
  resultado : list[dict[str, float]] 
    Variável que contém o resultado da função PWM, onde cada dicionário representa as probabilidades para cada base em uma posição
  
  
  Retorna
  -------------
  str 
    A Sequência mais provável dentro da Sequência dada
  '''
  
  dicionario = {}
  for subset in re.findall('(?=(....))', seq):
    dicionario[subset] = (prob_seq(subset, resultado))
  return max(dicionario, key=dicionario.get)


seq = "CATTGT"
mais_provavel = seq_provavel(seq, resultado)

print(f'A Sequência mais provável, dentro da Sequência {seq}, é {mais_provavel}.')

A Sequência mais provável, dentro da Sequência CATTGT, é CATT.


>### Função PSSM

A **função PSSM** (Position-Specific Scoring Matrix) é uma extensão da PWM. 

Enquanto a PWM representa a frequência relativa das bases, a PSSM atribui uma pontuação para cada base em cada posição, levando em consideração a probabilidade logarítmica da ocorrência de uma base em relação a uma distribuição de probabilidade de fundo.

In [66]:
import math

seqs = ['TGACTATACGTATGGTAGAT', 'ATCGTATACGTAGGTAGAC', 'TAGCTAGTCGTATGGTAGAT']

def pssm(seqs, pseudo = 1):

  """
  Calcula a Matriz de Pontuação de Posição Específica (PSSM) para um conjunto de sequências de DNA.

  Parâmetros
  -------------
  seqs : list[str]
    Lista de sequências de DNA.

  pseudo : float 
    Valor de pseudocount a ser adicionado para evitar problemas com probabilidades zero.


  Retorna
  -------------
  lista : list[dict[str, float]]
    Uma lista de dicionários que representa a PSSM.
  """

  bases = 'ATCG'
  lista = []

  for pos in list(zip(*seqs)):
    dicionario = {}
    for b in bases:
      # Fórmula da PSSM: log2((contagem da base + pseudocount) / (total de sequências + total de bases * pseudocount)) / 0.25
      dicionario[b] = math.log2((pos.count(b) + pseudo) / (len(seqs) + len(bases)*pseudo)) / 0.25

    lista.append(dicionario)
  return lista

matriz_pssm = pssm(seqs, pseudo = 1)

# O Table e o Headers já foram definidos para a função PWM

headers = ["Base"] + [f"Posição {i+1}" for i in range(len(seqs[0]))]
table = [[base] + [round(matriz_pssm[i][base], 3) for i in range(len(matriz_pssm))] for base in "ACGT"]
print(tabulate(table, headers))

Base      Posição 1    Posição 2    Posição 3    Posição 4    Posição 5    Posição 6    Posição 7    Posição 8    Posição 9    Posição 10    Posição 11    Posição 12    Posição 13    Posição 14    Posição 15    Posição 16    Posição 17    Posição 18    Posição 19
------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------
A            -7.229       -7.229       -7.229      -11.229      -11.229       -3.229      -11.229       -4.89       -11.229       -11.229       -11.229        -3.229       -11.229       -11.229       -11.229        -7.229        -4.89         -7.229        -4.89
C           -11.229      -11.229       -7.229       -4.89       -11.229      -11.229      -11.229      -11.229       -3.229       -11.229       -11.229       -11.229       -11.229       -11.229       -11.229  